# How to run a RL algorithm given a config file
- Step 1: Run ***load_model.py --file config_file*** to generate python files based on which we can automatically generate a GYM environment;
- Step 2: Run 